In [ ]:
"""
date : 2024-11-14
version : 1.0.0
writer : cwkang
"""

In [27]:
import os
import json
import logging
import argparse
import folium
import warnings
import numpy as np
import pandas as pd
from PIL import Image
from folium import plugins
from ast import literal_eval
from attrdictionary  import AttrDict

from utils import init_logger
from preprocessing import Preprocessing
from analyzer import clustering, metrix_to_sequence
from plot import CongestionPolt, CongestionImagePolt

logger = logging.getLogger(__name__)
warnings.filterwarnings("ignore")

In [2]:
init_logger()

cli_parser = argparse.ArgumentParser()
cli_args = cli_parser.parse_args("")

cli_args.config_dir = "config"
cli_args.config_file = "config.json"

with open(os.path.join(cli_args.config_dir, cli_args.config_file), encoding="utf-8") as f:
    args = AttrDict(json.load(f))

In [ ]:
%%time
prep = Preprocessing(args)

# SRCMAC Unique 값 추출
pre_raw_data = prep.get_preprocessed_data(raw_data=args.raw_data, time=True)
srcmac_unique_df = prep.calc_srcmac_unique(pre_raw_data)

# 일자별 SRCMAC의 수
people_cnt, _ = prep.calc_loc_people_counts(pre_raw_data, srcmac_unique_df, cumsum="00:01:00", loc=False, on_time=False, is_save=True)

# 시간대별 SRCMAC의 수
h_people_cnt, _ = prep.calc_loc_people_counts(pre_raw_data, srcmac_unique_df, cumsum="00:01:00", loc=False, on_time=True, is_save=True)

# 공간별 체류인원수
loc_srcmac_cnt, loc_people_cnt, srcmac_uni_matched_raw_data = prep.calc_loc_people_counts(pre_raw_data, srcmac_unique_df, cumsum="00:01:00", loc=True, on_time=False, is_save=True)

# 공간별 시간별 체류인원수
loc_srcmac_cnt, loc_h_people_cnt, srcmac_uni_matched_raw_data = prep.calc_loc_people_counts(pre_raw_data, srcmac_unique_df, cumsum="00:01:00", loc=True, on_time=True, is_save=True)

# Location 별 SRCMAC의 수 및 평균 Spenttime
h_loc_spenttime_mean_df, over_basetime_both_df = prep.calc_loc_people_spenttime(pre_raw_data, srcmac_unique_df, loc=True)

# Plot를 위한 Matrix 데이터
srcmac_loc_seq_metrix = prep.create_srcmac_loc_sequence_metrix(over_basetime_both_df)
cluster_metrix = clustering(srcmac_loc_seq_metrix)
mtx_seq_df = metrix_to_sequence(prep.zone_tbl, srcmac_loc_seq_metrix)

In [7]:
def generate_heatmap_points(start_poi, end_poi, num_poi=3):
    """
    경로에 따라 임의의 heatmap point 생성 함수
    두 지점 사이의 경로를 따라 점들과 가중치를 생성
    start_poi: 시작 지점 (위도, 경도)
    end_poi: 끝 지점 (위도, 경도)
    num_poi: 생성할 poi의 개수
    """
    latitudes = np.linspace(start_poi[0], end_poi[0], num_poi)
    longitudes = np.linspace(start_poi[1], end_poi[1], num_poi)
    counts = np.linspace(start_poi[2], end_poi[2], num_poi)
    
    return [[lat, lon, cnt] for lat, lon, cnt in zip(latitudes, longitudes, counts)]

In [ ]:
if args.task == "gangneung":
    # min: 43, max: 2527
    plot = CongestionPolt(prep.zone_tbl, args)

    if args.is_line_plot == True:
        # Line Plot    
        plot.seq_to_seq_flow_polyline(cluster_metrix, n_priority=50)

    if args.is_heatmap_plot == True:
        for hour in range(9, 21):
            # Heatmap
            plot.heat_map(loc_h_people_cnt, hour, is_interpolate=True)

elif args.task == "kme2024":
    # min: 26, max: 794
    plot = CongestionImagePolt(prep.zone_tbl, args)

    if args.is_line_plot == True:
        plot.seq_to_seq_flow_polyline(cluster_metrix, n_priority=50)

    if args.is_heatmap_plot == True: 
        for hour in range(10, 19):
            plot.heat_map(loc_h_people_cnt, hour, radius=70, is_interpolate=True, min_value=26, max_value=794)